In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [49]:
import load as pre
import numpy as np
import time
import pandas as pd

In [3]:
meta, y, mapping_dict = pre.load_data('../data/meta.json','../data/y.csv','../data/mappings.json')

In [4]:
a = pre.decode(meta,mapping_dict).Ranch.unique()






In [5]:
b = np.array([
    'VAP', 'OAP', 'OIB', 'OLB', 'VDG', 'SMB', 'OSF', 'SGB', 
    'SEB', 'TJC', 'ZCC', 'ZJL', 'ZJM', 'ZJZ', 'ZPR', 'ZVJ'
])

In [6]:
np.unique(np.concatenate([a,b]))

array(['MAP', 'OAP', 'OIB', 'OLB', 'OSF', 'SAB', 'SEB', 'SFB', 'SGB',
       'SGC', 'SJB', 'SLB', 'SMB', 'SMR', 'TJC', 'VAP', 'VDG', 'ZCC',
       'ZJL', 'ZJM', 'ZJZ', 'ZPR', 'ZVJ'], dtype=object)

In [56]:
finished_coordinates = {
    'MAP': (31.666431, -116.515308),
    'OAP': (31.920693, -116.243753),
    'OIB': (31.917818, -116.241634),  # estimated
    'OLB': (31.907724, -116.236947),
    'OSF': (31.925036, -116.244201),
    'SAB': (32.026187, -116.742880),
    'SEB': (32.017032, -116.689125),  # estimated
    'SFB': (32.025498, -116.753590),
    'SGB': (32.016094, -116.763115),
    'SGC': (32.016094, -116.763115),  # estimated
    'SJB': (32.022490, -116.754103),
    'SLB': (32.024978, -116.739198),
    'SMB': (31.907241, -116.241314),
    'SMR': (32.096734, -116.829672),
    'VAP': (31.421453, -116.269149),
    'VDG': (31.345217, -116.326015),
    'TJC': (25.091810, -111.315840),

}

In [53]:
grower_coordinates = {

    'TJC': (25.091810, -111.315840),
    'ZCC': (27.335660, -113.302957),  # estimated
    'ZJL': (28.301185, -114.028883),  # estimated - same as ZJM
    'ZJM': (28.301185, -114.028883),
    'ZJZ': (28.301185, -114.028883),  # estimated - same as ZJM
    'ZPR': (25.419456, -111.885169),
    'ZVJ': (28.286338, -113.994819)
}

In [8]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [58]:
#example pull:

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

vars = "temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,shortwave_radiation_sum,et0_fao_evapotranspiration,sunshine_duration,weather_code,precipitation_hours,wind_speed_10m_max"
var_list = vars.split(",")
for loc in grower_coordinates:
    lat = grower_coordinates[loc][0]
    lon = grower_coordinates[loc][1]
    start_date = "2012-01-01"
    end_date = "2025-06-17"
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
	"latitude": lat,
	"longitude": lon,
	"start_date": start_date,
	"end_date": end_date,
	"daily": vars
}
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    daily_data = pd.DataFrame(data = response.Daily().Variables(0).ValuesAsNumpy())
    daily_data["date"] = pd.date_range(
	start = pd.to_datetime(response.Daily().Time(), unit = "s", utc = True),
	end = pd.to_datetime(response.Daily().TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = response.Daily().Interval()),
	inclusive = "left"
)
    for var in var_list:
        daily_data[var] = response.Daily().Variables(var_list.index(var)).ValuesAsNumpy()
    daily_data.to_csv(f"../data/raw/daily_data_{loc}.csv")
    print(f"Saved {loc}")
    time.sleep(60)






Saved ZCC
Saved ZJL
Saved ZJM
Saved ZJZ
Saved ZPR
Saved ZVJ


In [59]:
loc = 'TJC'
lat = finished_coordinates[loc][0]
lon = finished_coordinates[loc][1]
start_date = "2012-01-01"
end_date = "2025-06-17"
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
"latitude": lat,
"longitude": lon,
"start_date": start_date,
"end_date": end_date,
"daily": vars
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]
daily_data = pd.DataFrame(data = response.Daily().Variables(0).ValuesAsNumpy())
daily_data["date"] = pd.date_range(
start = pd.to_datetime(response.Daily().Time(), unit = "s", utc = True),
end = pd.to_datetime(response.Daily().TimeEnd(), unit = "s", utc = True),
freq = pd.Timedelta(seconds = response.Daily().Interval()),
inclusive = "left"
)
for var in var_list:
    daily_data[var] = response.Daily().Variables(var_list.index(var)).ValuesAsNumpy()
daily_data.to_csv(f"../data/raw/daily_data_{loc}.csv")
print(f"Saved {loc}")

Saved TJC


check to make sure the things dont pull from overlapping lat/lon

example pull from every location, for loop

pull everything

save as json? so i can save it all as 1 file